In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
!pip install -U openmim && mim install "mmpretrain>=1.0.0rc8" && pip install -q lightly

Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.12.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.5/437.5 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 MB 952.2 kB/s eta 0:00:0000:010:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 54.8 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 2.6.2
    Uninstalling platformdirs-2.6.2:
      Successfully uninstalled platformdirs-2.6.2
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.0.0
    Uninstalling importlib-metadata-6.0.0:
      Successfully uninstalled importlib-metadata-6.0.0


In [10]:
model_names = ['SimCLR', 'BarlowTwins', 'Dino']
# datasets = ['TinyImageNet', 'CIFAR100', 'CIFAR10'] #, 'TinyImageNet']
datasets = ['TinyImageNet']

In [11]:
# DATASET = 'CIFAR10'
# DATASET = 'CIFAR100'
# DATASET = 'TinyImageNet'
N_EPOCHS = 15

In [12]:
from utils.model_utils import get_ssl_model
from utils.data_utils import get_train_loader, get_train_val_loaders
from utils.train_utils import reproducibility
from utils.train_utils import run_training_loop
from lightly.loss import NTXentLoss
import torch
from knn import KNN, knn_eval

from IPython.display import clear_output

In [13]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
criterion = NTXentLoss()

In [15]:
# main_loader = get_train_loader(dataset)
# train_loader, val_loader = get_train_val_loaders(dataset)
# model = get_ssl_model(model_name).to(device)
# knn_eval(model, train_loader, val_loader, device, transformer=model_name == 'Dino')

In [16]:
opt = dict()
lr = dict()
schedule = dict()

opt['Dino'] = torch.optim.Adam
lr['Dino'] = 3e-5
schedule['Dino'] = None
opt['SimCLR'] = torch.optim.SGD
lr['SimCLR'] = 0.006
schedule['SimCLR'] = {'scheduler':torch.optim.lr_scheduler.CyclicLR, 'base_lr':0.006, 'max_lr':0.01 }
opt['BarlowTwins'] = torch.optim.SGD
lr['BarlowTwins'] = 0.06
schedule['BarlowTwins'] = {'scheduler':torch.optim.lr_scheduler.CyclicLR, 'base_lr':0.006, 'max_lr':0.01 }

In [17]:
def get_scheduler(schedule_dict, opt):
    if schedule_dict is None:
        return None
    base_lr = schedule_dict['base_lr']
    max_lr = schedule_dict['max_lr']
    scheduler = schedule_dict['scheduler']
    return scheduler(opt, base_lr=base_lr, max_lr=max_lr)

In [ ]:
for model_name in model_names:
    for dataset in datasets:
        model = get_ssl_model(model_name).to(device)
        optim = opt[model_name](model.parameters(), lr=lr[model_name])
        scheduler = get_scheduler(schedule[model_name], optim)
        # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.85)
        # scheduler = torch.optim.lr_scheduler.CyclicLR(opt,base_lr=0.006, max_lr=0.01)
        main_loader = get_train_loader(dataset)
        train_loader, val_loader = get_train_val_loaders(dataset)
        knn_eval(model, train_loader, val_loader, device, transformer=model_name == 'Dino')
        run_training_loop(model, main_loader, criterion, optim, epochs=N_EPOCHS, device=device, scheduler=scheduler)
        train_acc, val_acc = knn_eval(model, train_loader, val_loader, device, transformer=model_name == 'Dino')
        torch.save(model.backbone.state_dict(), f'{model_name}_{dataset}_acc{val_acc:.2f}_epoch{N_EPOCHS}_lr{lr[model_name]}_finetune.pt')
        clear_output(wait=True)